# Vamos utilizar o mlflow para versionar os nossos modelos 

In [1]:
import mlflow

In [2]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression

In [3]:
ROOT_PATH = '../../data/'
SEED = 42
TARGET_COL = "Outcome"

## Utilizar o sqlite para fazer track das experiências

In [ ]:
mlflow.set_tracking_uri("sqlite:///../../mlruns/mlflow.db")

## Fazer criar e fazer set da experiência "Diabetes Prediction Experiment"

Com este novo tracking_uri, se fizermos apenas `mlflow.set_experiment(experiment_name="Diabetes Prediction Experiment")` como no notebook anterior os artifacts vão ser guardados numa pasta de `mlruns` dentro da pasta `logistic_regression`.

Para termos os artifacts dentro da pasta `mlruns` que está na root da pasta `rumos` temos que especificar explicitamente o `artifact_location` e usar primeiro o `create_experiment` com este parâmetro, e só depois fazer o `set_experiment`. O `set_experiment` faz internamente um `create_experiment` caso a experiência com aquele nome não exista, mas como não recebe o parâmetro `artifact_location` neste caso temos que chamar explicitamente o `create_experiment` 

## Criar os datasets

In [ ]:
train_path = ROOT_PATH + 'diabetes_train.csv'
test_path = ROOT_PATH + 'diabetes_test.csv'

train_set = pd.read_csv(train_path)
test_set = pd.read_csv(test_path)

## Criar uma run

## Guardar datasets, modelos, artefactos, métricas e parametros da run - e **registar modelo**

In [ ]:
X_train = train_set.drop([TARGET_COL], axis = 1)
y_train = train_set[TARGET_COL]

X_test = test_set.drop([TARGET_COL], axis = 1)
y_test = test_set[TARGET_COL]

In [ ]:
scaler = StandardScaler()

features_names = X_train.columns

X_train[features_names] = scaler.fit_transform(X_train)
X_test[features_names] = scaler.transform(X_test)

In [ ]:
X_train.head()

In [ ]:
X_train.head()

In [ ]:
logistic_reg = LogisticRegression(random_state = SEED).fit(X_train, y_train)

In [ ]:
y_preds = logistic_reg.predict(X_test)

In [ ]:
accuracy_score(y_test, y_preds)

In [ ]:
logistic_reg = LogisticRegression(random_state = SEED)

parameters = {'C': [0.001,0.01,0.1,1,10,100]}

clf = GridSearchCV(logistic_reg, parameters, cv = 5).fit(X_train, y_train)

In [ ]:
tunned_logistic_reg = clf.best_estimator_

In [ ]:
tunned_logistic_reg.get_params()

In [ ]:
y_preds = tunned_logistic_reg.predict(X_test)

In [ ]:
acc = accuracy_score(y_test, y_preds)

## Terminar a run

## Opcional - Ver o modelo registado na UI do mlflow

A UI do mlflow permite ver de forma visual todos os modelos registados até ao momento, qual é a útlima versão do mesmo e a que runs estão associados.

Para correr a UI do mflow, tendo nós o modelo registado na db, temos que especificar esta db como a "backend store" em que temos os metados.

Para correr a UI do mflow é necessário executar, na Anaconda Prompt na raiz deste projeto (pasta rumos) e tendo activo o ambiente utilizado neste projeto, o comando:

`mlflow ui --backend-store-uri sqlite:///./mlruns/mlflow.db`

**Nota:** O comando em cima irá iniciar a UI de mlflow na porta 5000. Caso queiram mudar esta porta devem acrescentar `--port <PORT>` ao comando (em que <PORT> deve ser substituido pela porta desejada). 

O comando acima não irá funcionar caso tenham tido alguns problemas no Windows com a instalação do mlflow, mas como o título desta secção indica este passo é apenas opcional e não irá ser avaliado.

Após executarem este comando, vão poder ver a UI do mlflow no vosso browser acedendo a 

`localhost:5000`

(se tiverem alterado a porta em que o mlflow UI é iniciado então devem de alterar também aqui o 5000 por essa porta)

Na tab de `Models` podem explorar os modelos que registaram.